<a href="https://colab.research.google.com/github/max-moore/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS18_212_Regression_2_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

# Train/Test Split

In [ ]:
# Quick look at the dataframe

df

In [ ]:
# Converting the 'created' feature to datetime for easy conditional usage

df['created'] = pd.to_datetime(df['created'])

In [ ]:
# Creating a train/test split

train = df[df['created'] < '2016-06']
test = df[df['created'] >= '2016-06']

In [ ]:
# Checking the shape, looks good

train.shape, test.shape

((31844, 34), (16973, 34))

# Feature Engineering

In [ ]:
# I want to engineer a 'total_rooms' feature

df['total_rooms'] = df['bedrooms'] + df['bathrooms']

In [ ]:
df

In [ ]:
# Engineering a 'cats and dogs allowed' feature

df['cats_and_dogs'] = df['cats_allowed'] * df['dogs_allowed']

In [ ]:
# Checking, this lines up with our dataframe's 'cat/dog allowed' values

df['cats_and_dogs']

0        0
1        1
2        0
3        0
4        0
        ..
49347    0
49348    1
49349    1
49350    1
49351    0
Name: cats_and_dogs, Length: 48817, dtype: int64

# Fit Linear Regression Model

In [ ]:
# Import estimator class and instantiate

from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [ ]:
# Choosing target and features

target = 'price'

y_train = train[target]
y_test = test[target]

features = ['longitude',
            'latitude']

print(f'Linear Regression, dependent on: {features}')

X_train = train[features]
X_test = test[features]

Linear Regression, dependent on: ['longitude', 'latitude']


In [ ]:
# Fit the model

model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Model's Coefficients and Intercepts

In [ ]:
print('Model Coefficients:\n',
      model.coef_, '\n',
      'Model Intercept:\n',
      model.intercept_)

Model Coefficients:
 [-16215.70541389   2208.18971896] 
 Model Intercept:
 -1285931.9851655466


# Regression Metrics

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from matplotlib.patches import Rectangle

model = LinearRegression()

features = ['longitude', 'latitude']
target = ['price']

X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Getting our line of best fit using our training data
y_pred = model.predict(X_train)

# Grabbing and printing regression metrics for train split
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

Root Mean Squared Error: 1704.2079552598782
Mean Absolute Error: 1147.1493278231853
R^2: 0.06463820907125062


In [ ]:
# Getting our line of best fit using our training data
y_pred = model.predict(X_test)

# Grabbing and printing regression metrics for test split
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

Root Mean Squared Error: 1703.0806186783316
Mean Absolute Error: 1139.700457630829
R^2: 0.06677485649195314


In [ ]:
model = LinearRegression()

features = ['longitude', 'latitude', 'cats_allowed', 'dogs_allowed']
target = ['price']

X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Getting our line of best fit using our training data
y_pred = model.predict(X_train)

# Grabbing and printing regression metrics for train split
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

Root Mean Squared Error: 1702.5292057284573
Mean Absolute Error: 1144.5776675147295
R^2: 0.06648007909991249


In [ ]:
# Getting our line of best fit using our training data
y_pred = model.predict(X_test)

# Grabbing and printing regression metrics for test split
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

Root Mean Squared Error: 1701.1104062130744
Mean Absolute Error: 1137.8470965787908
R^2: 0.06893281456667122
